<img src="Images/cover_readme.png" alt="Example Image" style="width:1400px; height:400px;" >

--- 
# **DeepCellMap Laborarory - Find best tissue segmentation adapted to each dataset and images**
---

Depending on the dataset and the type of data, tissue segmentation can be done using different steps. In this notebook, differents steps are tested. Results are saved and visualised thanks to an html file. 



In [1]:
# This imports your common setup code from grabber.py
from grabber import * 
%precision 2
%load_ext autoreload
%autoreload 2 

# import pandas as pd

from config.dataset_management import take_config_from_dataset

from utils.util import *
from utils.util_fig_display import *
from utils.util_colors_drawing import *

from preprocessing import filter, slide, tiles

from stat_analysis import deep_cell_map
from segmentation_classification import segmentation,classification
from segmentation_classification.classification import ModelClassification, segment_classify_cells_wsi

%matplotlib inline 


Pb importing pylibCZIrw
Pb importing pylibCZIrw


## Dataset selection 

In [2]:
display(dataframe_dropdown)
dataset_name = dataframe_dropdown.value

Dropdown(description='Select dataset:', options={'generated_rois': 'generated_rois', 'ihc_microglia_fetal_huma…

In [3]:
dataset_name = dataframe_dropdown.value
dataset_config = take_config_from_dataset(dataset_name)
print(blue("We are playing with the dataset: {}".format(dataset_config.dataset_name)))


We are playing with the dataset: ihc_microglia_fetal_human_brain_database


# 1. Possible steps to segment tissue

Methods are all defined in filter.apply_process_from_name(). 

In [4]:
param_best_cellpose =dataset_config.param_best_cellpose
step_possibilities = dict({
            "cellpose" : param_best_cellpose,

            #Binarization steps 
            "multi_otsu_thresholding" :  ["dont_consider_background", 40],
            "otsu_thresholding" : None,
            "manual_threshold" : 10,
            "rgb_threshold_based_binarization_microglia_IHC": None,

            #Morphological operations 
            "erosion": 2,
            "dilation" : 4,
            "dilation2":3,
            "opening": 5,

            #Other operations 
            "fill_holes" : None,
            "filter_center_cells" : dataset_config.tile_width,
            "remove_small_objects" : 150,
            "remove_large_objects" : 10000,
            "rgb_to_eosin" : None,
            "take_largest_component" : 1,
        })

# 2. Definition of the different steps

The different tissue segmentation steps can be common to all the images but can also be specified for each image.  

Once a good sequence of steps works for many images, the dictionnary of the different steps can be given as value to the key *default* of the parameter **dataset_config.tissue_segmentation_param** 

**If this shared sequence of steps doesn't work for an image N**, a new sequence can be added to the dictionnary **tissue_segmentation_param** specifying the key as *image_N*

*This is shared with the tissue segmentation method* 

In [12]:
sequence_processing = dict({
    "manual_threshold" : 244,
    "dilation" : 4,#30
    "fill_holes" : None,
    "remove_small_objects" : 300
    })


image_list = [1]

#Define segmentor
segmentor = segmentation.ModelSegmentation(dataset_config=dataset_config, object_to_segment = "tissue", tissue_segmentation_param=sequence_processing)

#Get images to segment 
list_images, list_images_names  = slide.get_images_to_segment(image_list,dataset_config)

#Apply segmentation based on param and save results in html 
list_images_processed, list_process_names = segmentor.test_segmentation_grayscale(list_images,sequence_processing=sequence_processing)

# segmentor.laboratory_segmentation_save_images(list_images_processed, list_process_names,image_list=image_list)
segmentation.ModelSegmentation.laboratory_segmentation_save_images(dataset_config,list_images_processed, list_process_names, segmentation_type = "Tissue",image_list=image_list)

HTML saved at /Users/U1029063/Documents/ens_project/Code_reviewers_new_submision/output/ihc_microglia_fetal_human_brain_database/1_image_pre_processing/Tissue_filtering_laboratory/img_01/06_test_segmentation.html


# 3. Add new segmentation steps to the configuration file and re apply "tissue_segmentation_step" in Notebook 1 ! 

Just add a new key "image_<n_slide>" to the dictionnary self.tissue_segmentation_param bellow "default" 
